In [1]:
import pandas as pd
from datetime import datetime
import math
from utils.scalability_utils import cluster_to_dataframe
from models.SSPMF import SSPMFModel
from models.JGSMF import solve_with_phases
from models.JGSMF import solve_with_constant_bins as solve_JGSMF
from models.JGSMF_1 import solve_with_constant_bins as solve_JGSMF_1
from models.JGSMF_2 import solve_with_constant_bins as solve_JGSMF_2
from models.JGSMF_3 import solve_with_constant_bins as solve_JGSMF_3

In [2]:
cluster_directory = "../data/MTSP/Laporte/Tabela4/"

df = cluster_to_dataframe(cluster_directory)

df = df.sample(frac=1).reset_index(drop=True)

In [3]:
print(df['num_tools'].unique())
print(df['magazine_capacity'].unique())


[25 15 20]
[10 20 15  5]


In questo dataset tutti le istanze hanno 9 job e un numero di tools tra 15, 20 o 25. La capienza invece è tra 5, 10, 15 o 20.
Per l'analisi di scalabilità ho deciso di prendere tutte le combinazioni di numero di tools e capienza, per poi valutarne la velocità di esecuzione tra i 2 modelli

In [4]:
scalability_df = df.drop_duplicates(subset=['num_tools', 'magazine_capacity'])

scalability_df

,filename,num_jobs,num_tools,magazine_capacity,job_tools
0,L13-5.txt,9,25,10,"{1: [3, 16, 17, 20, 22, 23], 2: [2, 10, 16, 18..."
1,L19-6.txt,9,25,20,"{1: [2, 3, 4, 5, 9, 10, 13, 14, 16, 17, 18, 22..."
4,L2-8.txt,9,15,10,"{1: [10, 12, 13, 14], 2: [5, 13], 3: [2, 5], 4..."
9,L7-6.txt,9,20,15,"{1: [7, 8, 9, 11, 12, 13, 14, 15], 2: [2, 5, 6..."
12,L5-4.txt,9,20,10,"{1: [1, 6, 7, 10, 14, 20], 2: [2, 5, 6, 7, 13,..."
16,L30-3.txt,9,25,5,"{1: [2, 8, 16, 21, 22], 2: [9, 10, 13, 20], 3:..."
17,L32-5.txt,9,25,15,"{1: [13, 22], 2: [5, 6, 11, 17, 24], 3: [1, 4,..."
45,L27-6.txt,9,20,5,"{1: [2, 3, 6, 7], 2: [12, 17, 18, 19], 3: [9, ..."
52,L1-8.txt,9,15,5,"{1: [3, 8, 9, 13], 2: [1, 5, 6, 9, 10], 3: [7,..."


Ogni esperimento su queste istanze verrà ripetuto 5 volte per ottenenere la mediana del tempo di ogni combinazione. Poi verrà costruito il grafico della scalability con questi dati

In [ ]:
results = []

#SSPMF Model
print("SSPMF Model")
for index,row in scalability_df.iterrows():

    print("Instance: ", row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    median_time = 0

    for i in range(0, 5):
        start_time = datetime.now()
        SSPMF_model = SSPMFModel(jobs, tools, magazine_capacity, job_tools_requirements)
        SSPMF_model.optimize()
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    median_time = pd.Series(same_iteration_times).median()
    results.append(["SSPMF",num_jobs, num_tools, magazine_capacity, median_time])

print("-----------------------------------")
#JGSMF Model
print("JGSMF Model")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    median_time = 0

    for i in range(0, 5):
        start_time = datetime.now()
        results = solve_with_phases(jobs, tools, magazine_capacity, job_tools_requirements, time_limit=3600)
        if results is not None:
            job_order, switches = results
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    median_time = pd.Series(same_iteration_times).median()
    results.append(["JGSMF",num_jobs, num_tools, magazine_capacity, median_time])

results_df = pd.DataFrame(results, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])

In [6]:
results_df

,Model,num_jobs,num_tools,magazine_capacity,time
0,SSPMF,9,15,5,0.961002
1,SSPMF,9,25,20,0.448999
2,SSPMF,9,25,10,23.447193
3,SSPMF,9,25,15,12.244513
4,SSPMF,9,20,15,5.336124
5,SSPMF,9,15,10,0.041000
6,SSPMF,9,20,10,11.876751
7,SSPMF,9,25,5,5.867588
8,SSPMF,9,20,5,0.366002
9,JGSMF,9,15,5,2.128522


Dato che alcuni esperimenti sembrano più veloci di quanto dovrebbero (considerando le combinazioni di parametri), la cui rapidità è probabilmente dovuto ai tools richiesti da ogni job .Ora ripeterò l'esperimento eseguito sopra solo considerando 5 diverse istanze per combinazione ed eseguendone poi la mediana sulla mediana dei tempi, in modo da cercare di mitigare questo problema

In [7]:
new_scalability_df = df.groupby(['num_tools', 'magazine_capacity']).head(5)


new_scalability_df

,filename,num_jobs,num_tools,magazine_capacity,job_tools
0,L13-5.txt,9,25,10,"{1: [3, 16, 17, 20, 22, 23], 2: [2, 10, 16, 18..."
1,L19-6.txt,9,25,20,"{1: [2, 3, 4, 5, 9, 10, 13, 14, 16, 17, 18, 22..."
2,L22-6.txt,9,25,20,"{1: [3, 4, 5, 15, 18, 22, 24], 2: [2, 4, 6, 10..."
3,L23-5.txt,9,25,20,"{1: [2, 3, 5, 10, 11, 12, 16, 21], 2: [3, 6, 8..."
4,L2-8.txt,9,15,10,"{1: [10, 12, 13, 14], 2: [5, 13], 3: [2, 5], 4..."
5,L4-2.txt,9,15,10,"{1: [2, 3, 4, 5, 6, 8, 11, 12, 13, 15], 2: [3,..."
6,L20-6.txt,9,25,20,"{1: [20, 21, 24], 2: [15, 16], 3: [4, 8, 12, 1..."
7,L2-3.txt,9,15,10,"{1: [4, 5, 10, 11, 12], 2: [10, 15], 3: [2, 10..."
8,L4-8.txt,9,15,10,"{1: [3, 5, 6, 7, 8, 12], 2: [4, 5, 8, 12, 13, ..."
9,L7-6.txt,9,20,15,"{1: [7, 8, 9, 11, 12, 13, 14, 15], 2: [2, 5, 6..."


In [ ]:
new_results_df = []
# SSPMF Model
print("SSPMF Model")
for (num_tools, magazine_capacity), group in new_scalability_df.groupby(['num_tools', 'magazine_capacity']):
    same_iteration_times = []
    print("Group ", num_tools, magazine_capacity)

    for index, row in group.iterrows():
        print("Instance", row['filename'])
        num_jobs = row['num_jobs']
        job_tools_requirements = row['job_tools']

        jobs = [x for x in range(1, num_jobs + 1)]
        tools = [x for x in range(1, num_tools + 1)]

        iteration_times = []

        for i in range(0, 5):
            start_time = datetime.now()
            SSPMF_model = SSPMFModel(jobs, tools, magazine_capacity, job_tools_requirements)
            SSPMF_model.optimize()
            end_time = datetime.now()
            elapsed_time = (end_time - start_time).total_seconds()
            iteration_times.append(elapsed_time)

        median_time = pd.Series(iteration_times).median()
        same_iteration_times.append(median_time)

    overall_median_time = pd.Series(same_iteration_times).median()
    new_results_df.append(["SSPMF", num_jobs, num_tools, magazine_capacity, overall_median_time])

print("-----------------------------------")
# JGSMF Model
print("JGSMF Model")
for (num_tools, magazine_capacity), group in new_scalability_df.groupby(['num_tools', 'magazine_capacity']):
    same_iteration_times = []
    print("Group ", num_tools, magazine_capacity)

    for index, row in group.iterrows():
        print("Instance", row['filename'])
        num_jobs = row['num_jobs']
        job_tools_requirements = row['job_tools']

        jobs = [x for x in range(1, num_jobs + 1)]
        tools = [x for x in range(1, num_tools + 1)]

        iteration_times = []

        for i in range(0, 5):
            start_time = datetime.now()
            results = solve_with_phases(jobs, tools, magazine_capacity, job_tools_requirements, time_limit=3600)
            if results is not None:
                job_order, switches = results
            end_time = datetime.now()
            elapsed_time = (end_time - start_time).total_seconds()
            iteration_times.append(elapsed_time)

        median_time = pd.Series(iteration_times).median()
        same_iteration_times.append(median_time)

    overall_median_time = pd.Series(same_iteration_times).median()
    new_results_df.append(["JGSMF", num_jobs, num_tools, magazine_capacity, overall_median_time])

new_results_df = pd.DataFrame(new_results_df, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])


JGSMF Model
Group  15 5
Instance L1-8.txt
Trying Phase 1 with 6 bins...
Phase 1 found a solution with 6 bins and 12 switches.
Trying Phase 2 with 7 bins...
Found less or equal switches in phase 2 with 7 bins and 12 switches.
Checking in phase 3...
Solution found.
Trying Phase 1 with 6 bins...
Phase 1 found a solution with 6 bins and 12 switches.
Trying Phase 2 with 7 bins...
Found less or equal switches in phase 2 with 7 bins and 12 switches.
Checking in phase 3...
Solution found.
Trying Phase 1 with 6 bins...
Phase 1 found a solution with 6 bins and 12 switches.
Trying Phase 2 with 7 bins...
Found less or equal switches in phase 2 with 7 bins and 12 switches.
Checking in phase 3...
Solution found.
Trying Phase 1 with 6 bins...
Phase 1 found a solution with 6 bins and 12 switches.
Trying Phase 2 with 7 bins...
Found less or equal switches in phase 2 with 7 bins and 12 switches.
Checking in phase 3...
Solution found.
Trying Phase 1 with 6 bins...
Phase 1 found a solution with 6 bins and

In [ ]:
new_results_df

Esperimenti sulle 4 diverse implementazioni di JGSMF, ottenute aggiungendo di volta in volta diversi cuts constraint. Nello specifico:
- JGSMF -> no symmetry/performances cuts
- JGSMF_1 -> symmetry cuts
- JGSMF_2 -> performance cuts senza cliques
- JGSMF_3 -> aggiunte anche le cliques

Inoltre trattandosi di tanti esperimenti, verrà utilizzato lo scalabity_df contenente una istanza per combinazione di parametri del cluster A (Tabela 4)

In [ ]:
JGSMF_results = []
print("JGSMF Model")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    median_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        results = solve_JGSMF(jobs, tools, magazine_capacity, job_tools_requirements, num_bins,time_limit=3600)
        if results is not None:
            job_order, switches = results
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    median_time = pd.Series(same_iteration_times).median()
    JGSMF_results.append(["JGSMF",num_jobs, num_tools, magazine_capacity, median_time])
print("-----------------------------------")
print("JGSMF Model 1")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    median_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        results = solve_JGSMF_1(jobs, tools, magazine_capacity, job_tools_requirements, num_bins,time_limit=3600)
        if results is not None:
            job_order, switches = results
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    median_time = pd.Series(same_iteration_times).median()
    JGSMF_results.append(["JGSMF_1",num_jobs, num_tools, magazine_capacity, median_time])
print("-----------------------------------")
print("JGSMF Model 2")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    median_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        results = solve_JGSMF_2(jobs, tools, magazine_capacity, job_tools_requirements, num_bins,time_limit=3600)
        if results is not None:
            job_order, switches = results
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    median_time = pd.Series(same_iteration_times).median()
    JGSMF_results.append(["JGSMF_2",num_jobs, num_tools, magazine_capacity, median_time])
print("-----------------------------------")
print("JGSMF Model 3")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    median_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        results = solve_JGSMF_3(jobs, tools, magazine_capacity, job_tools_requirements, num_bins,time_limit=3600)
        if results is not None:
            job_order, switches = results
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    median_time = pd.Series(same_iteration_times).median()
    JGSMF_results.append(["JGSMF_3",num_jobs, num_tools, magazine_capacity, median_time])

JGSMF_results_df = pd.DataFrame(JGSMF_results, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])
